In [1]:
!pip install spotipy
!pip install pandas
!pip install scikit-learn
!pip install bokeh

import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, CategoricalColorMapper
from bokeh.layouts import column
from bokeh.models.widgets import Div

In [2]:
# Storing my Spotify API credentials - delete before commit!
CLIENT_ID = '349309ae5ef349eb85096837bc86547b'
CLIENT_SECRET = '2c5d4880a00a4bba80c915b55ccb54b5'

# Authenticate with Spotify
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
csv_file = '50_tracks.csv'
field_names = ['Artist Name', 'Artist URI', 'Track Name', 'Track URI', 'Album',
               'Key', 'Mode', 'Danceability', 'Energy', 'Loudness', 'Speechiness',
               'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Embed Code']


In [4]:
with open(csv_file, 'w', newline='') as file: # Writing to the csv, with open, using writer
    writer = csv.DictWriter(file, fieldnames=field_names)
    writer.writeheader()

In [5]:
def artist_lookup():
    # Prompt the user for an artist name
    user_artist = input("Enter an artist: ")

    # Search for the artist using Spotify API
    results = sp.search(q='artist:' + user_artist, type='artist', limit=1)

    # Get the artist URI from the search results
    if results['artists']['items']:
        artist = results['artists']['items'][0]
        artist_uri = artist['uri']
        top_tracks(artist_uri, user_artist) # Passing to top_tracks, with artist_uri and user_artist as arguments
        related_artists(artist_uri) # Passing to related_artists, with artist_uri as argument
    else:
        print("Artist not found.")

In [6]:
def top_tracks(artist_uri, user_artist):
    results = sp.artist_top_tracks(artist_uri) # Saving as results
    tracks = results['tracks'] # Looking for just tracks

    print("\n\nThe top 10 tracks for the artist are...\n")
    with open(csv_file, 'a', newline='', encoding="utf-8") as file: # Opening and appending the csv now
        writer = csv.DictWriter(file, fieldnames=field_names)
        for track in tracks:
            track_uri = track['uri'][14:] # Dropping the first 15 characters (spotify:tracks:) from the output
            track_info = sp.track(track_uri) # Getting information for each track, such as Album Name
            audio_features = sp.audio_features(track_uri) # Getting audio features

            # Perform Spotify oEmbed lookup
            oembed_api_url = (f'https://open.spotify.com/oembed?url=https://open.spotify.com/track/{track_uri}&format=json')
            response = requests.get(oembed_api_url)

            if response.status_code == 200:
                oembed_data = response.json()
                embedded_html = oembed_data['html'] # Taking just the html portion of the response.json()
            else:
                embedded_html = ''

            writer.writerow({ # Writing everything to the csv now
                'Artist Name': user_artist,
                'Artist URI': artist_uri[15:],
                'Track Name': track['name'],
                'Track URI': track_uri,
                'Album': track_info['album']['name'],
                'Key': audio_features[0]['key'],
                'Mode': audio_features[0]['mode'],
                'Danceability': audio_features[0]['danceability'],
                'Energy': audio_features[0]['energy'],
                'Loudness': audio_features[0]['loudness'],
                'Speechiness': audio_features[0]['speechiness'],
                'Acousticness': audio_features[0]['acousticness'],
                'Instrumentalness': audio_features[0]['instrumentalness'],
                'Liveness': audio_features[0]['liveness'],
                'Valence': audio_features[0]['valence'],
                'Tempo': audio_features[0]['tempo'],
                'Embed Code': embedded_html
            })

            print(f"{track['name']}")

In [7]:
def related_artists(artist_uri): # Using this to look up the 4 most closely related artists for the user_artist
    print("\nAnd 4 related artists... ")
    related = sp.artist_related_artists(artist_uri)
    related_artists_uris = []
    for artist in related['artists'][:4]: # Limiting to just 4
        print(artist['name'])
        artist_uri = artist['uri']
        related_artists_uris.append(artist_uri) # Appending to the list
        top_tracks(artist_uri, artist['name']) # Running each of these through top_tracks

In [8]:
artist_lookup()

Enter an artist: PROFF


The top 10 tracks for the artist are...

Farewell To The Moon - PROFF Remix
Surge - PROFF & Igor Garanin Remix (Mixed)
Nibbana - Volen Sentir's Pink Sky Retouch
Consequence Of You
Light Magic
Whale Song
Dark Magic
Surge - PROFF & Igor Garanin Remix
What Have You Done To Me? - Jody Wisternoff, James Grant & PROFF Remix (Mixed)
For The Night

And 4 related artists... 
Jaytech


The top 10 tracks for the artist are...

Future Story - Notaker Remix
All In - Rodg Remix
Meant To Be (ABGT526) - Jaytech Remix
Tachyon
Meant To Be - Jaytech Remix
Meant To Be (ABGT525) - Jaytech Remix
Gray Horizon (feat. Melody Gough)
Zulu [ABGT185]
Metawave - Jaytech Remix
All In - Edit
Mat Zo


The top 10 tracks for the artist are...

Colours
Closure
Easy
Easy - Radio Edit
Astatine
Reflections
Hodo - Mat Zo: Redo
Adrian
Games
Closure (ABGT531)
Boom Jinx


The top 10 tracks for the artist are...

Breathing (ASOT 1122)
Breathing (ABGT530)
Breathing
Forgotten September
The Dark (ABGT523) -

In [9]:
df = pd.read_csv('50_tracks.csv', encoding='utf-8')

df.sample(5)

,Artist Name,Artist URI,Track Name,Track URI,Album,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Embed Code
18,Jaytech,5mpJ1Z8BYEOG0rrdO4a2nG,Metawave - Jaytech Remix,4cHCMlrB2pu4nka58KhG4C,Metawave (Jaytech Remix),7,1,0.567,0.779,-10.732,0.0332,0.000334,0.606000,0.214,0.0715,126.003,"<iframe style=""border-radius: 12px"" width=""100..."
3,PROFF,3jAosRBCdrybxqTnrI4Sld,Consequence Of You,3qJjesE755cmOz5w4LBhYc,Consequence Of You,11,1,0.696,0.878,-6.949,0.0677,0.007110,0.591000,0.140,0.1650,123.016,"<iframe style=""border-radius: 12px"" width=""100..."
19,Jaytech,5mpJ1Z8BYEOG0rrdO4a2nG,All In - Edit,0M7LAJeo73n4Wltz7Uo1T4,Fatum Presents: 20 Years Of Anjunabeats,0,0,0.508,0.666,-6.562,0.0337,0.008450,0.000169,0.146,0.0771,128.060,"<iframe style=""border-radius: 12px"" width=""100..."
22,Mat Zo,2n7USVO8fO8FF8zq4kG2N1,Easy,12XS8xxdJpQIYmenVvTSOY,Damage Control,2,1,0.548,0.797,-6.407,0.0466,0.001630,0.000209,0.339,0.3240,127.998,"<iframe style=""border-radius: 12px"" width=""100..."
45,Nitrous Oxide,0UJQXkmx6GjqUxugf7zzkR,Children Of The Sun (ABGT526),2YBdGjBkFtZXVx0mIZdChq,Group Therapy 526,3,0,0.580,0.692,-8.550,0.0536,0.001070,0.778000,0.125,0.4000,126.008,"<iframe style=""border-radius: 12px"" width=""100..."


In [10]:
data_columns = df.columns[7:-1]  # Exclude the last column "Embed Code"

# Normalizing with Min-Max scaling - this will take tempo and convert it into a 0-1 value
scaler = MinMaxScaler()
df[data_columns] = scaler.fit_transform(df[data_columns])

In [11]:
variances = df[data_columns].var()

# Sort the fields based on their variances in descending order
sorted_variances = variances.sort_values(ascending=False)
print("All of the variances:")
print(sorted_variances)

# Select the top two fields with the highest variances
top_two_fields = sorted_variances.head(2)
field1 = top_two_fields.index[0]  # Get the field name of the first field
field2 = top_two_fields.index[1]  # Get the field name of the second field

print(f"The two fields with the greatest variance are {field1} and {field2}.\n")

All of the variances:
Instrumentalness    0.129373
Liveness            0.060955
Valence             0.057647
Loudness            0.035830
Danceability        0.032938
Speechiness         0.031363
Energy              0.026256
Tempo               0.022680
Acousticness        0.021051
dtype: float64
The two fields with the greatest variance are Instrumentalness and Liveness.



In [12]:
# Ask the user if they want to use the fields with the greatest variance or define their own fields
user_choice = input("Would you like select your own categories to analyze? (Y/N): ")

if user_choice.lower() == 'y':
    field1 = input("Enter Field 1: ") # Would be nice to incorporate some input filtering/control here
    field2 = input("Enter Field 2: ")

Would you like select your own categories to analyze? (Y/N): Y
Enter Field 1: Instrumentalness
Enter Field 2: Danceability


In [13]:
# Get unique artists
artists = df['Artist Name'].unique()

# Defining a color palette for each artist
color_palette = ['#FF0000', '#FF8000', '#000080', '#008000', '#00FFFF']

# Create a color mapper mapping each artist to a color
num_artists = len(artists)
palette = color_palette[:num_artists]
color_mapper = CategoricalColorMapper(factors=artists, palette=palette) # Building using CCM

# Create the scatterplot with colored points
scatterplot = figure()
scatterplot.circle(x=field1, y=field2, source=df, fill_color={'field': 'Artist Name', 'transform': color_mapper}, size=6, legend_field='Artist Name')

GlyphRenderer(id='p1057', ...)

In [14]:
# Customize hover tooltips
hover = HoverTool(tooltips=[("Artist", "@{Artist Name}"), ("Track", "@{Track Name}")])
scatterplot.add_tools(hover)

# Set plot attributes
scatterplot.xaxis.axis_label = field1
scatterplot.yaxis.axis_label = field2

# Get the HTML data from the last field for each record
html_data = df.iloc[:, -1]

# Create a Div element to display the HTML data     
html_content = Div(text='', width=800, height=200)

# Concatenate all the HTML data and update the Div element
html_content.text = '\n\n'.join(html_data)

# Create a layout with the scatterplot and the Div element
layout = column(scatterplot, html_content)

# Show the plot within the Jupyter Notebook
from bokeh.io import output_notebook, show
output_notebook()
show(layout)

Loading BokehJS ...